In [1]:
from dask.distributed import Client, LocalCluster

cluster = LocalCluster(n_workers=4,ncores=4)
client = Client(cluster)

In [2]:
client

Client Scheduler: tcp://127.0.0.1:45641 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 4.03 GB


In [3]:
#Loading a single file
import dask.dataframe as dd
import dask.array as da

import pandas as pd
weekly_df = pd.read_csv('M4DataSet/Weekly-train.csv')
# weekly_df_array = weekly_df.values
# weekly_df_array = da.transpose(weekly_df_array)
# weekly_df = dd.dataframe(weekly_df_array)
weekly_df = weekly_df.set_index('V1').T
weekly_df.head()


V1,W1,W2,W3,W4,W5,W6,W7,W8,W9,W10,...,W350,W351,W352,W353,W354,W355,W356,W357,W358,W359
V2,1089.20,195.928,258.7,3460.0,4696.0,728.0,5721.0,3010.0,9643.0,246.98,...,13099.0,747.0,5174.0,1593.0,4549.0,4798.0,4923.0,4401.0,5977.0,3965.0
V3,1078.91,194.796,259.8,3340.0,4765.0,650.0,5783.0,3600.0,10243.0,222.30,...,13099.0,747.0,5585.0,2199.0,5264.0,5192.0,6382.0,4955.0,7770.0,5237.0
V4,1079.88,192.710,260.2,2770.0,4144.0,755.0,5757.0,3910.0,10935.0,449.98,...,10202.0,2218.0,4445.0,1668.0,3621.0,4194.0,4387.0,3680.0,5843.0,4304.0
V5,1063.58,190.288,260.8,2520.0,3945.0,829.0,6198.0,3640.0,9798.0,449.98,...,13099.0,2350.0,5274.0,1725.0,3968.0,4217.0,4979.0,4214.0,6169.0,4265.0
V6,1060.61,188.770,261.1,2740.0,4733.0,831.0,6038.0,4150.0,9179.0,449.98,...,10202.0,2300.0,3900.0,1718.0,3774.0,3926.0,4660.0,4139.0,5990.0,4488.0


In [4]:
weekly_df.tail()

V1,W1,W2,W3,W4,W5,W6,W7,W8,W9,W10,...,W350,W351,W352,W353,W354,W355,W356,W357,W358,W359
V2594,NaN,NaN,NaN,1932.91,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
V2595,NaN,NaN,NaN,2056.49,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
V2596,NaN,NaN,NaN,1984.55,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
V2597,NaN,NaN,NaN,2004.56,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
V2598,NaN,NaN,NaN,2385.81,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
weekly_df_indexed = weekly_df.copy()
weekly_df_indexed.index.name = 'Time'
weekly_df_indexed.head()

V1,W1,W2,W3,W4,W5,W6,W7,W8,W9,W10,...,W350,W351,W352,W353,W354,W355,W356,W357,W358,W359
Time,,,,,,,,,,,,,,,,,,,,,
V2,1089.20,195.928,258.7,3460.0,4696.0,728.0,5721.0,3010.0,9643.0,246.98,...,13099.0,747.0,5174.0,1593.0,4549.0,4798.0,4923.0,4401.0,5977.0,3965.0
V3,1078.91,194.796,259.8,3340.0,4765.0,650.0,5783.0,3600.0,10243.0,222.30,...,13099.0,747.0,5585.0,2199.0,5264.0,5192.0,6382.0,4955.0,7770.0,5237.0
V4,1079.88,192.710,260.2,2770.0,4144.0,755.0,5757.0,3910.0,10935.0,449.98,...,10202.0,2218.0,4445.0,1668.0,3621.0,4194.0,4387.0,3680.0,5843.0,4304.0
V5,1063.58,190.288,260.8,2520.0,3945.0,829.0,6198.0,3640.0,9798.0,449.98,...,13099.0,2350.0,5274.0,1725.0,3968.0,4217.0,4979.0,4214.0,6169.0,4265.0
V6,1060.61,188.770,261.1,2740.0,4733.0,831.0,6038.0,4150.0,9179.0,449.98,...,10202.0,2300.0,3900.0,1718.0,3774.0,3926.0,4660.0,4139.0,5990.0,4488.0


### Cleaning the data 

In [6]:
import numpy as np
def clean_ts(ts):
    
    """
    takes time series as input and returns cleaned time series
    
    *removes outliers
    *linearly interpolates missing values
    
    returns time series back
    """
    #find the start of the time series
    start_ts = ts[ts.notna()].index[0]
    #find the end of the time series
    end_ts = ts[ts.notna()].index[-1]
    ts = ts.loc[start_ts:end_ts]
    
    #sets outliers to NaN defined as a point that is 3 deviations away from rolling mean + 3*rolling standard deviation
    ts = ts.where(~(abs(ts) > (ts.rolling(center=False,window=3).mean() + (ts.rolling(center=False,window=3,min_periods = 3).std() * 3))),np.nan)
    #linearly interpolate any NaN values
    ts = ts.interpolate()
    #make sure any remaining null values are filled with 0's
    ts = ts.fillna(0)
            
    return ts

In [7]:

clean_ts(weekly_df_indexed['W20']).head()

Time
V2    3477.0
V3    3473.0
V4    3474.0
V5    3473.0
V6    3755.0
Name: W20, dtype: float64

In [8]:
#Taking only 20 series 
weekly_df_small = weekly_df_indexed.iloc[:,:20]
weekly_df_small.head()

V1,W1,W2,W3,W4,W5,W6,W7,W8,W9,W10,W11,W12,W13,W14,W15,W16,W17,W18,W19,W20
Time,,,,,,,,,,,,,,,,,,,,
V2,1089.20,195.928,258.7,3460.0,4696.0,728.0,5721.0,3010.0,9643.0,246.98,1449.24,1148.0,730.386,737.736,219.05,219.05,1739.35,532.95,6704.0,3477.0
V3,1078.91,194.796,259.8,3340.0,4765.0,650.0,5783.0,3600.0,10243.0,222.30,1450.67,1141.0,738.346,741.652,203.96,203.96,1739.36,532.98,6967.0,3473.0
V4,1079.88,192.710,260.2,2770.0,4144.0,755.0,5757.0,3910.0,10935.0,449.98,1228.98,1134.0,743.049,743.255,210.91,210.91,1727.58,533.00,6838.0,3474.0
V5,1063.58,190.288,260.8,2520.0,3945.0,829.0,6198.0,3640.0,9798.0,449.98,1069.18,1127.0,736.319,734.559,187.09,187.09,1727.58,533.00,6364.0,3473.0
V6,1060.61,188.770,261.1,2740.0,4733.0,831.0,6038.0,4150.0,9179.0,449.98,1279.67,1120.0,730.393,729.651,178.13,178.13,1748.18,512.40,6032.0,3755.0


### Single Worker for loop to clean the series

In [9]:
%%time
output = []
for series in weekly_df_small:
    cleaned_series = clean_ts(weekly_df_small[series])
    output.append(cleaned_series)
    
#turn list of series back into full dataframe
full_df = pd.concat(output,ignore_index=False,keys=weekly_df_small.columns,axis=1)
# #sort by index to get back to format we want
sorted_index = full_df.index.to_series().str[1:].astype(int).sort_values()
cleaned_df = full_df.reindex(index=sorted_index.index)


CPU times: user 164 ms, sys: 0 ns, total: 164 ms
Wall time: 229 ms


/home/pintu/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  import sys


### parallelized for loop using dask.delayed to clean the series (4 workers)

In [10]:
import dask

In [11]:
%%time
output = []
for series in weekly_df_small:
    cleaned_series = dask.delayed(clean_ts)(weekly_df_small[series])
    output.append(cleaned_series)
    
total = dask.delayed(output).compute()
#turn list of series back into full dataframe
full_df = pd.concat(total,ignore_index=False,keys=weekly_df_small.columns,axis=1)

sorted_index = full_df.index.to_series().str[1:].astype(int).sort_values()
cleaned_df = full_df.reindex(index=sorted_index.index)

CPU times: user 132 ms, sys: 12 ms, total: 144 ms
Wall time: 438 ms


/home/pintu/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


The rule of thumb  with dask is it's worth it over pandas if our computation time start taking in the order of minutes, for example longer than 20 minutes.

In [12]:
cleaned_df.head()

V1,W1,W2,W3,W4,W5,W6,W7,W8,W9,W10,W11,W12,W13,W14,W15,W16,W17,W18,W19,W20
V2,1089.20,195.928,258.7,3460.0,4696.0,728.0,5721.0,3010.0,9643.0,246.98,1449.24,1148.0,730.386,737.736,219.05,219.05,1739.35,532.95,6704.0,3477.0
V3,1078.91,194.796,259.8,3340.0,4765.0,650.0,5783.0,3600.0,10243.0,222.30,1450.67,1141.0,738.346,741.652,203.96,203.96,1739.36,532.98,6967.0,3473.0
V4,1079.88,192.710,260.2,2770.0,4144.0,755.0,5757.0,3910.0,10935.0,449.98,1228.98,1134.0,743.049,743.255,210.91,210.91,1727.58,533.00,6838.0,3474.0
V5,1063.58,190.288,260.8,2520.0,3945.0,829.0,6198.0,3640.0,9798.0,449.98,1069.18,1127.0,736.319,734.559,187.09,187.09,1727.58,533.00,6364.0,3473.0
V6,1060.61,188.770,261.1,2740.0,4733.0,831.0,6038.0,4150.0,9179.0,449.98,1279.67,1120.0,730.393,729.651,178.13,178.13,1748.18,512.40,6032.0,3755.0


## Forecasting

Now we can play with this dataset. We have processed it in parallel.